In [40]:
#!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple/
#!pip3 install --user pyarma -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [41]:
from pyarma import *
import numpy as np

In [42]:
def partial_forward(pW,pV,V,alpha,beta,memory,prev_stimulus,prev_memory,der,time):
    for ii in range(memory.n_rows):
        temp1 = zeros( memory.n_rows,prev_stimulus.n_rows)
        temp2 = zeros( memory.n_rows,memory.n_rows)
        for cc in range(memory.n_rows):
            temp1 = temp1 + V[ii,cc] * pW[(ii,time)]
            temp2 = temp2 + V[ii,cc] * pV[(ii,time)]
        add = zeros( size(temp1))
        add[ii,:] = prev_stimulus
        pW[(ii,time+1)] = der[ii]*( beta*add + alpha*temp1 )
        add = zeros( size(temp2) )
        add[ii,ii] = prev_memory[ii]
        pV[(ii,time+1)] = der[ii]*alpha*( repmat(prev_memory,memory.n_rows,1)+ temp2 )
    return pW,pV

In [43]:
def partial_update(prev_stimulus,prev_memory,stimulus,pW,pV,V,position ):
    W_real = repmat( prev_stimulus,stimulus.n_rows,1 )
    V_real = repmat( prev_memory,stimulus.n_rows,1 )
    W_imag = zeros(stimulus.n_rows,prev_stimulus.n_rows)
    V_imag = zeros(stimulus.n_rows,stimulus.n_rows)
    for ii in range(stimulus.n_rows):
            temp1 = zeros(1,prev_stimulus.n_rows)
            temp2 = zeros(1,stimulus.n_rows)
            for cc in range(stimulus.n_rows):
                W_imag[ii,:] = temp1 + W[ii,cc] * pW[(cc,len(pW)-position-2)][:,ii].eval()
                #W_imag[ii,:] = temp1 + W[ii,cc] * pW[(cc,0)][:,ii].eval()
                V_imag[ii,:] = temp2 + V[ii,cc] * pV[(cc,len(pV)-position-2)][:,ii].eval()
                #V_imag[ii,:] = temp2 + V[ii,cc] * pV[(cc,0)][:,ii].eval()
    return W_real,W_imag,V_real,V_imag

In [44]:
def complexReLU(x,order):
    y = zeros( size(x) )
    der = zeros( 1, y[1,:].n_rows );
    #temp = sum(x^2)
    if order == 0:                  
        y = x
        y = [ y , x ]
    elif order == 1 :
        y = 1 / ( 1+exp( x )  )
        y = [ y , y @ (1 - y) ]
    return y

In [45]:
XTrain = mat(40, 5);
YTrain = mat(40, 1);
output_dim = 1;
input_dim = 5;
iteration = 2;
timestep = 3;
alpha = 1;
beta = 1;
eta = 0.01;
order = 0;

In [46]:
W = 2*randn(output_dim,input_dim) - 1;   
V = 2*randn(output_dim,output_dim) - 1;    

W_update = zeros(size(W));
V_update = zeros(size(V));

memory_value = 2*randn(1,output_dim)-1;

partialW = {}
partialV = {}

In [47]:
for jj in range(output_dim):
    partialW[(jj,0)] = zeros(output_dim,input_dim)
    partialV[(jj,0)] = zeros(output_dim,output_dim)

In [48]:
A = size(XTrain)
n = A[0]
sampling = n * iteration

In [53]:
error_train = {}
for kk in range(sampling):
    sample =  kk % n 
    if sample == 0:
        sample = n
    if sample - timestep >= 0:
        #print(sample-timestep+1)
        X = XTrain[ sample-timestep:sample-1,: ]
        Y = YTrain[ sample-timestep:sample-1,: ]
    else:
        X = XTrain[ 0:sample-1,: ]
        Y = YTrain[ 0:sample-1,: ]
    pW = partialW
    pV = partialV
    overallError = 0 
    out_diff =mat()
    for time in range(Y.n_rows):
        inp = X[time,:].eval()
        ins = mat(2,1)
        ins[0,:] = alpha*inp*W.t() - beta*memory_value[size(memory_value)[0]-1,:]*V
        ins[1,:] = beta*inp*W.t() + alpha*memory_value[size(memory_value)[0]-1,:]*V.t()
        [feed,der] = complexReLU(ins,0)
        out = feed[0,:];
        out_ad = der;
        memory = feed[1,:]
        memory_value =  join_vert(memory_value,memory)
        prev_memory = memory_value[size(memory_value)[0]-2,:]
        [pW,pV] = partial_forward(pW,pV,V,alpha,beta,memory,inp,prev_memory,der,time)
        ###########
        error_out = Y[time,:] - out
        out_diff =  join_vert(out_diff, error_out @ out_ad)
        overallError = overallError + 0.5*sum(error_out*error_out)
        #############
    for time in range(Y.n_rows):
        print(Y.n_rows)
        position = Y.n_rows-1-time
        print(position)
        inpt = X[size(X)[0]-1-position,:].eval()
        prev_memory = memory_value[size(memory_value)[0]-1-position-1,:].eval()
        hid_diff = out_diff[size(out_diff)[0]-1-position,:].eval()
        [W_real,W_imag,V_real,V_imag] = partial_update(inpt,prev_memory,out,pW,pV,V,position ) 
        delta_W = hid_diff.t() @ ( alpha * W_real - beta * W_imag )
        delta_V = hid_diff.t() @ ( - beta * ( V_real + V_imag ) )
        
        W_update = W_update + delta_W
        V_update = V_update + delta_V
    W = W + W_update * eta
    V = V + V_update * eta
    
    W_update = W_update * 0
    V_update = V_update * 0
 
    error_train[kk] = overallError
Memory = memory_value[size(memory_value)[0]-1,:]
#Partial_W = pW[(0,len(pW)-1)]
#Partial_V = pV[(0,len(pV)-1)]
for jj in range(output_dim):
    partialW[(jj,0)] = pW[(jj,len(pW)-1)]
    partialV[(jj,0)] = pW[(jj,len(pW)-1)]

3
2
3
1
3
0
1
0
2
1
2
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
1
0
2
1
2
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0
3
2
3
1
3
0


In [50]:
print(pW)

{(0, 0): <pyarma.pyarma.mat object at 0x5589cc0b50e0>
[matrix size: 1x5]
        0        0        0        0        0
, (0, 1): <pyarma.pyarma.mat object at 0x5589cbcee2d0>
[matrix size: 1x5]
        0        0        0        0        0
, (0, 2): <pyarma.pyarma.mat object at 0x5589cc0c6b60>
[matrix size: 1x5]
        0        0        0        0        0
, (0, 3): <pyarma.pyarma.mat object at 0x5589cc0b50e0>
[matrix size: 1x5]
        0        0        0        0        0
}


In [51]:
print(error_train)

{0: <pyarma.pyarma.mat object at 0x5589cc0c9580>
[matrix size: 1x1]
   0.0350
, 1: <pyarma.pyarma.mat object at 0x5589cc0ae140>
[matrix size: 1x1]
   2.9375e-07
, 2: <pyarma.pyarma.mat object at 0x5589cbd06b50>
[matrix size: 1x1]
   6.1359e-09
, 3: <pyarma.pyarma.mat object at 0x5589cbce2b10>
[matrix size: 1x1]
   2.5720e-12
, 4: <pyarma.pyarma.mat object at 0x5589cc040860>
[matrix size: 1x1]
   2.2059e-17
, 5: <pyarma.pyarma.mat object at 0x5589cc0636f0>
[matrix size: 1x1]
   1.8919e-22
, 6: <pyarma.pyarma.mat object at 0x5589cc0637c0>
[matrix size: 1x1]
   1.6226e-27
, 7: <pyarma.pyarma.mat object at 0x5589cbce2be0>
[matrix size: 1x1]
   1.3916e-32
, 8: <pyarma.pyarma.mat object at 0x5589cc063bd0>
[matrix size: 1x1]
   1.1935e-37
, 9: <pyarma.pyarma.mat object at 0x5589cba6f900>
[matrix size: 1x1]
   1.0236e-42
, 10: <pyarma.pyarma.mat object at 0x5589cc040790>
[matrix size: 1x1]
   8.7793e-48
, 11: <pyarma.pyarma.mat object at 0x5589cbcee200>
[matrix size: 1x1]
   7.5296e-53
, 12: <

In [52]:
print(memory_value[0:100,:])

[matrix size: 101x1]
   1.8303e+00
  -2.6182e-01
   3.7452e-02
            :            
   6.5693e-85

